# Analysis of Heat Wave Definitions

In [ ]:
### Import all required libraries

import xarray as xr
import dask
import numpy as np
from scipy.stats import norm
import pymannkendall as mk
import matplotlib.pyplot as plt
# import cartopy.crs as ccrs
from cartopy import feature, crs
import matplotlib.animation as animation
from matplotlib import rc
from IPython.display import HTML
import os
from cdo import *
cdo = Cdo()
import warnings
warnings.filterwarnings('ignore')
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import glob
from varname import nameof

In [ ]:
## Specify path to the file
path='/media/kenz/1B8D1A637BBA134B/CHIRTS'
# cdo = Cdo('~/media/kenz/1B8D1A637BBA134B/CHIRTS/scripts/tmp/')

In [ ]:
### specify file names

tmax = 'Tmax/chirts.Tmax.1983.2016.WA.days_p25.nc'
tmin = 'Tmin/chirts.Tmin.1983.2016.WA.days_p25.nc'

tmax90 = 'Tmax/chirts.Tmax90.1983.2016.WA.days_p25.nc'
tmin90 = 'Tmin/chirts.Tmin90.1983.2016.WA.days_p25.nc'

tmean = 'Tmean/chirts.Tmean.1983.2016.WA.days_p25.nc'
tmean95 = 'Tmean/chirts.clim.Tmean95.1983.2016.WA.days_p25.nc'
# Tmean/chirts.clim.Tmean95.1983.2016.WA.days_p25.nc

tmean_accl = 'Tmean/accl.nc'
tmean_sig = 'Tmean/sig95.nc'

tmean_ehf = 'Tmean/ehf.nc'

tmx = 'Tmax/tx.nc'
tmn = 'Tmin/tn.nc'

In [ ]:
### Defining functions  ###

def set_fig_params(ax):
    for i,j in enumerate(ax):
    # for i in (range(0,len(axes))):
        ax[i].set_extent([-19,15,4,24])
        ax[i].add_feature(feature.COASTLINE)
        ax[i].add_feature(feature.BORDERS)
        ax[i].add_feature(feature.STATES, linewidth = 0.2)
        ax[i].set_xticks([-20,-10,0,10], crs=crs.PlateCarree())
        ax[i].set_yticks([5,10,15,20], crs=crs.PlateCarree())
        lon_formatter = LongitudeFormatter(zero_direction_label=True)
        lat_formatter = LatitudeFormatter()
        ax[i].xaxis.set_major_formatter(lon_formatter)
        ax[i].yaxis.set_major_formatter(lat_formatter)
def sens_slope(data):
    data = data.groupby('time.year').mean('time')
    data = data.sel(longitude=np.arange(-19.875, 21.875,0.25), latitude=np.arange(3.125,26.875,0.25), method = 'nearest')
    output=[]
    for i in np.arange(len(data.latitude.values)):
        for j in np.arange(len(data.longitude.values)):
            try:
                slope_val = mk.sens_slope(data[:,i,j]).slope
            except:
                slope_val = np.nan
            output.append(slope_val)

    output = np.copy(output).reshape(data.latitude.size,data.longitude.size)
    slopes=xr.DataArray(output, dims=('latitude','longitude'), coords={'latitude':data.latitude,'longitude':data.longitude})
    return slopes

def rising_filter(array, axis):

    # Make sure there are enough points
    assert(array.shape[axis] == 5)
    # Make sure we're working on the last axis
    assert(axis == array.ndim-1 or axis == -1)
    
    left = array[..., 1]
    right = array[..., 2:].sum(axis=axis)

    return np.logical_and(np.isnan(left), np.isfinite(right))

def rising_filter_dask(x, dim):

    return xr.apply_ufunc(rising_filter, x, input_core_dims=[[dim]],
                             kwargs={'axis': -1},
                             dask='parallelized',
                             output_dtypes=[bool])
def HWD(data):
    # s = data>0
    # s = s.drop_duplicates('time')
    # candidates = tx.where(s)
    windows = data.chunk({'time':20}).rolling(time=5, center=True, min_periods=1).construct('rolling_dim')
    heatwave_starts = rising_filter_dask(windows, dim='rolling_dim')
    return heatwave_starts


In [ ]:
def sens_slopes(data):
    data = data.sel(longitude=np.arange(-19.875, 21.875,0.25), latitude=np.arange(3.125,26.875,0.25), method = 'nearest')
    output=[]
    for i in np.arange(len(data.latitude.values)):
        for j in np.arange(len(data.longitude.values)):
            try:
                slope_val = mk.sens_slope(data[:,i,j]).slope
            except:
                slope_val = np.nan
            output.append(slope_val)

    output = np.copy(output).reshape(data.latitude.size,data.longitude.size)
    slopes=xr.DataArray(output, dims=('latitude','longitude'), coords={'latitude':data.latitude,'longitude':data.longitude})
    return slopes

In [ ]:
### More Functions ###
# def df_read(path,file,var):
#     ds = xr.open_dataset(f'{path}/{file}')[f'{var}']
#     # [f'{temperature_type}']
#     return ds
def df_read(path,file,var):
    ds = xr.open_dataset(f'{path}/{file}')[f'{var}'].sortby('time').drop_duplicates('time').sel(time=slice('1983',None))
    return ds
def sellonlat(data):
    return data.sel(longitude=10,latitude=10,method='nearest')
def selone(data):
    return data.sel(time='2010')
def mask_temp(path,data,pctl):
    sub = cdo.sub(input = ' '.join((f'{path}/{data}', f'{path}/{pctl}')))
    temp = df_read(path,data,str(data[:4])).chunk({'time':5000})
    extreme = xr.open_dataset(sub)[f'{data}'[:4]].drop_duplicates('time')
    data = temp.where(extreme>0)
    return data
def remove_temp_files():
    for i in glob.glob('/tmp/cdo*'):
        os.remove(i)

# def HWF(path,file, temperature_type='Tmax',var='Tmax'):
#     if temperature_type not in ['Tmax', 'Tmin']:
#         raise ValueError("Invalid temperature_type. Use 'Tmax' or 'Tmin'.")
#     path,file, temperature_type = path,file, temperature_type
#     ds = df_read(path,file,var)
#     ds = HWD(ds).sel(time=slice('1983', '2016')).sum('time') / 30
#     ds = cal_ct90(path, temperature_type=temperature_type).groupby('time.year').count('time').mean('year')*ds
#     return ds

# def HWF_ehf(path, file, var):
#     ds = df_read(path, file,var).sortby('time').chunk({'time':5})
#     mask = ds.where(ds>0, drop = True)
#     ehf_hwd = mask.rolling(time=3).sum().sel(time=slice('1983', '2016')).mean('time')
#     ehf = abs(ds.mean('time'))
#     return ehf*ehf_hwd


In [ ]:
# accl = df_read(path,tmean_accl,'Tmax').chunk({'time':2700})
# sig = df_read(path,tmean_sig,'Tmax').chunk({'time':2700})
# accl = xr.where(accl<=1,1,accl)
# sig = xr.where(sig<0,0,sig)
# ehf = accl*sig
# ehf = ehf.where(ehf>0)
# ds_gtehf = ehf.groupby('time.year').count()

In [ ]:
# ehf.to_netcdf('Tmean/ehf.nc')

In [ ]:
ehf = df_read(path,tmean_ehf,'Tmax')
ds_gtehf = ehf.groupby('time.year').count()

In [ ]:
ehf_f = ehf.chunk({'time':-1,'longitude':20,'latitude':20})

In [ ]:
%%time
hwf_ehf = ehf_f.groupby('time.year').sum().mean('year')
hwf_ehf = hwf_ehf.compute()

In [ ]:
remove_temp_files()

In [ ]:
# tx = mask_temp(path,tmax,tmax90)
# tn = mask_temp(path,tmin,tmin90)

# tx.to_netcdf('Tmax/tx.nc')
# tn.to_netcdf('Tmain/tn.nc')

tx = df_read(path, tmx, 'Tmax')
tn = df_read(path, tmn, 'Tmin')

In [ ]:
ds_gtx = tx.groupby('time.year').count('time')

In [ ]:
ds_gtn = tn.groupby('time.year').count('time')

In [ ]:
%%time
windows = tx.chunk({'time':20}).rolling(time=5, center=True, min_periods=1).construct('rolling_dim')
hwn_tx = rising_filter_dask(windows, dim='rolling_dim')

windows = tn.chunk({'time':20}).rolling(time=5, center=True, min_periods=1).construct('rolling_dim')
hwn_tn = rising_filter_dask(windows, dim='rolling_dim')

#### HWN for Maximum Temperature

In [ ]:
selone(sellonlat(df_read(path,tmax,'Tmax'))).plot(label = 'Tmax', alpha = 0.2)
selone(sellonlat(df_read(path,tmax90,'Tmax'))).plot(label = 'Tmax90', alpha = 0.7)
selone(sellonlat(tx)).plot(label ='peaks', color='green')
plt.legend()
plt.title('2010 \n')
# plt.savefig(f'{path}/figures/Tem and Peaks.jpeg')

In [ ]:
levels = range(int(ds_gtx.max()))
fig, axes = plt.subplots(figsize=[7,5])#ncols=1) #Creating the basis for the plot
def animate(time):
    ds_gtx.isel(year=time).plot(levels=levels, ax=axes, add_colorbar= False)

ani = animation.FuncAnimation(fig, animate, ds_gtx.year.size, interval=400, blit=False)

mld= ds_gtx.isel(year=0).plot.contourf(levels=levels, ax=axes, add_colorbar= False)
cbar= fig.colorbar(mld)
cbar.set_label('CTx90pctl')
fig.suptitle("Tmax Yearly Heatwave Days", fontsize= 18)

HTML(ani.to_jshtml())

In [ ]:
ani.save('figures/Tmax_10_day_window_avearged_yearly_heatwave_events.gif', writer='imagemagick', fps = 2) #Save animation as gif-file

#### HWN for Minimum Temperature

In [ ]:
selone(sellonlat(df_read(path,tmin,'Tmin'))).plot(label = 'Tmin', alpha = 0.2)
selone(sellonlat(df_read(path,tmin90,'Tmin'))).plot(label = 'Tmin90', alpha=0.7)
selone(sellonlat(tn)).plot(label ='peaks', color='green')
plt.legend()
plt.title('2010')
# plt.savefig(f'{path}/figures/Tmin and Peaks.jpeg')

In [ ]:
# # ds_tn_gt_tn90 = mask_temp(path,tmin,tmin90)
# ds_tn_gt_tn90 = tn
# ds_gtn = ds_tn_gt_tn90.groupby('time.year').count('time')
# ds_gtn = ds_gtn.load()

In [ ]:
levels = range(int(ds_gtn.max()))
fig, axes = plt.subplots(figsize=[7,5])#ncols=1) #Creating the basis for the plot
def animate(time):
    ds_gtn.isel(year=time).plot(levels=levels, ax=axes, add_colorbar= False)

ani = animation.FuncAnimation(fig, animate, ds_gtn.year.size, interval=400, blit=False)

mld= ds_gtn.isel(year=0).plot.contourf(levels=levels, ax=axes, add_colorbar= False)
cbar= fig.colorbar(mld)
cbar.set_label('CTn90pctl')
fig.suptitle("Tmin Yearly Heatwave Days", fontsize= 18)

HTML(ani.to_jshtml())

In [ ]:
ani.save(f'{path}/figures/Tmin_10_day_window_avearged_yearly_heatwave_events.gif', writer='imagemagick', fps = 2) #Save animation as gif-file

#### Calculating HWN for EHF

In [ ]:
### calculate this once to store and open

# yearmax = cdo.yearmax(input = f'{path}/{tmean}')
# yearmin = cdo.yearmin(input = f'{path}/{tmean}')
# cdo.yearpctl('95',input = "{} {} {}".format(f'{path}/{tmean}', yearmin, yearmax),output ='Tmean/chirts.clim.Tmean95.1983.2016.WA.days_p25.nc')

# cdo.runmean('3',input = f'{path}/{tmean}',output = 'Tmean/r3_95.nc')
# cdo.runmean('30', input = f'{path}/{tmean}',output = 'Tmean/r30_95.nc')
# cdo.sub(input="{} {}".format('Tmean/r3_95.nc', 'Tmean/r30_95.nc'), output='Tmean/accl.nc')
# cdo.sub(input="{} {}".format('/Tmean/r3_95.nc', f'{path}/{tmean95}'), output='Tmean/sig95.nc')

In [ ]:
# cdo.sub(input="{} {}".format(f'{path}/Tmean/r3_95.nc', f'{path}/{tmean95}'), output='Tmean/sig95.nc')

In [ ]:
# accl = df_read(path,tmean_accl,'Tmax').chunk({'time':5000})
# sig = df_read(path,tmean_sig,'Tmax').chunk({'time':5000})
# accl = xr.where(accl<=1,1,accl)
# sig = xr.where(sig<0,0,sig)
# ehf = accl*sig
# ehf = ehf.where(ehf>0)

In [ ]:
# ds_gtehf = ehf.groupby('time.year').count()

In [ ]:
levels = range(int(ds_gtehf.max()))
fig, axes = plt.subplots(figsize=[7,5])#ncols=1) #Creating the basis for the plot
def animate(time):
    ds_gtehf.isel(year=time).plot(levels=levels, ax=axes, add_colorbar= False)

ani = animation.FuncAnimation(fig, animate, ds_gtehf.year.size, interval=400, blit=False)

mld= ds_gtehf.isel(year=0).plot.contourf(levels=levels, ax=axes, add_colorbar= False)
cbar= fig.colorbar(mld)
cbar.set_label('EHF')
fig.suptitle("EHF Yearly Extreme Temperature days", fontsize= 18)

HTML(ani.to_jshtml())

In [ ]:
ani.save(f'{pathfigures/EHF_95_Tmean_day_window_avearged_yearly_heatwave_events.gif', writer='imagemagick', fps = 2) #Save animation as gif-file

#### Calculating Heatwave Number (HWN)  #####

 - Heatwaves are defined as a period of `at least three (3) days` where for each day the Maximum or Minimum temperature is in the top 10% for that day of the year.
 - **NB**: EHF is already in a three (3) average therefore satisfying `consecutive three (3) days` requirement of a heatwave.

In [ ]:
%%time

# Create subplots
fig, ax = plt.subplots(ncols=3, nrows=2, figsize=(10, 5), subplot_kw={'projection': crs.PlateCarree()})
ax = ax.flatten()
set_fig_params(ax)

# Plot data
vmax=15
# cc = 'gist_heat_r'# plt.savefig(f'{path}/figures/Tem and Peaks.jpeg')
cc='YlOrBr'
### cb1 = ds_gtx.mean('year').plot(ax=ax[0], cmap=cc, vmax=60, add_colorbar=False)
# cb1 = ds_gtx.where(ds_gtx>0).mean('year').plot(ax=ax[0], cmap=cc, vmax=55, vmin=35, add_colorbar=False)

hwdtx = hwn_tx.sum('time')/33
cb1 = hwdtx.where(hwdtx>0).plot(ax=ax[0], cmap=cc, vmax=vmax, add_colorbar=False)
ax[0].set_title('CTx90 1983-2016')

### ds_gtn.mean('year').plot(ax=ax[1], cmap=cc, vmax=60, add_colorbar=False)
# ds_gtn.where(ds_gtn>0).mean('year').plot(ax=ax[1], cmap=cc, vmax=55,  vmin=35, add_colorbar=False)

hwdtn = (hwn_tn.sum('time')/33)
hwdtn.where(hwdtn>0).plot(ax=ax[1], cmap=cc, vmax=vmax, add_colorbar=False)
ax[1].set_title('CTn90 1983-2016')

ds_gtehf.where(ds_gtehf>0).mean('year').plot(ax=ax[2], cmap=cc, vmax=vmax, add_colorbar=False)
ax[2].set_title('EHF 1983-2016')

cm = 'RdBu_r'
vm = 1

# cb2 = sens_slopes(ds_gtx.where(ds_gtx>0).load()).plot(ax=ax[3], vmax=vm, cmap=cm, add_colorbar=False)
# ax[3].set_title('CTx90')


txsl = hwn_tx.groupby('time.year').sum('time')
# txsl = txsl.where(txsl>0).load()
cb2 = sens_slopes(txsl.where(txsl>0).load()).plot(ax=ax[3], vmax=vm, cmap=cm, add_colorbar=False)

tnsl = hwn_tn.groupby('time.year').sum('time')
sens_slopes(tnsl.where(tnsl>0).load()).plot(ax=ax[4], vmax=vm, cmap=cm, add_colorbar=False)

sens_slopes(ds_gtehf.where(ds_gtehf>0).load()).plot(ax=ax[5], cmap=cm, vmax=vm, add_colorbar=False)
ax[5].set_title('EHF')

# # cb = [cb1, cb2]
label = ['HWN','Trends']

# cax2 = fig.add_axes([1,0.62,0.02,0.3])
# fig.colorbar(cb3,cax = cax2, orientation='vertical', extend='both',label='ehf')
cb = [cb1, cb2]
# Add colorbars
for i, j in enumerate([0.62, 0.13]):
    cax = fig.add_axes([1, j, 0.02, 0.3])
    fig.colorbar(cb[i], cax=cax, orientation='vertical', extend='both', label = label[i])

fig.tight_layout()
# plt.savefig(path+'/figures/Climatologies of Yearly Heatwaves.jpeg', bbox_inches='tight')

In [ ]:
#### Decadal plots on HWN  ######

In [ ]:
%%time
t1 = ['1984', '1994', '2004']
t2 = ['1993', '2003', '2013']
for i, j in enumerate(t1): 
    # Create subplots
    fig, ax = plt.subplots(ncols=3, nrows=2, figsize=(10, 5), subplot_kw={'projection': crs.PlateCarree()})
    ax = ax.flatten()
    set_fig_params(ax)

    # Plot data
    vmax=15
    cc='coolwarm'

    hwdtx = hwn_tx.sel(time=slice(t1[i],t2[i])).sum('time')/10
    cb1 = hwdtx.where(hwdtx>0).plot(ax=ax[0], cmap=cc, vmax=vmax, add_colorbar=False)
    ax[0].set_title(f'CTx90 {t1[i]}-{t2[i]}')
    # break

    hwdtn = hwn_tn.sel(time=slice(t1[i],t2[i])).sum('time')/10
    hwdtn.where(hwdtn>0).plot(ax=ax[1], cmap=cc, vmax=vmax, add_colorbar=False)
    ax[1].set_title(f'CTn90 {t1[i]}-{t2[i]}')

    ds_gtehf.where(ds_gtehf>0).sel(year=slice(t1[i],t2[i])).mean('year').plot(ax=ax[2], cmap=cc, vmax=vmax, add_colorbar=False)
    ax[2].set_title(f'EHF {t1[i]}-{t2[i]}')

    cm = 'RdBu_r'
    vm = 1

    txsl = hwn_tx.groupby('time.year').sum('time').sel(year=slice(t1[i],t2[i]))
    # txsl = txsl.where(txsl>0).load()
    cb2 = sens_slopes(txsl.where(txsl>0).load()).plot(ax=ax[3], vmax=vm, cmap=cm, add_colorbar=False)
    ax[3].set_title(f'CTx90 {t1[i]}-{t2[i]}')


    tnsl = hwn_tn.groupby('time.year').sum('time').sel(year=slice(t1[i],t2[i]))
    sens_slopes(tnsl.where(tnsl>0).load()).plot(ax=ax[4], vmax=vm, cmap=cm, add_colorbar=False)
    ax[4].set_title(f'CTn90 {t1[i]}-{t2[i]}')

    sens_slopes(ds_gtehf.where(ds_gtehf>0).sel(year=slice(t1[i],t2[i])).load()).plot(ax=ax[5], cmap=cm, vmax=vm, add_colorbar=False)
    ax[5].set_title(f'EHF {t1[i]}-{t2[i]}')

    # # cb = [cb1, cb2]
    label = ['HWN','Trends']

    # cax2 = fig.add_axes([1,0.62,0.02,0.3])
    # fig.colorbar(cb3,cax = cax2, orientation='vertical', extend='both',label='ehf')
    cb = [cb1, cb2]
    # Add colorbars
    for i, j in enumerate([0.62, 0.13]):
        cax = fig.add_axes([1, j, 0.02, 0.3])
        fig.colorbar(cb[i], cax=cax, orientation='vertical', extend='both', label = label[i])

    fig.tight_layout()
    plt.savefig(f'{path}/figures/Climatologies of Yearly Heatwaves {t1[i]}-{t2[i]}.jpeg', bbox_inches='tight')

In [ ]:
##### Seasonal Averaged HWN ######
# %%time #hwntx
def hwn_plt(data,t):
    %%time
    fig, ax = plt.subplots(ncols=4, nrows=3, figsize=(10, 5), subplot_kw={'projection': crs.PlateCarree()})
    ax = ax.flatten()
    set_fig_params(ax)
    t1 = ['1984', '1994', '2004']
    t2 = ['1993', '2003', '2013']
    for i, j in enumerate(t1):

        # Plot data
        vmax=3
        cc='coolwarm'

        hwntx = data.sel(time=slice(t1[i],t2[i])).groupby('time.season').sum('time')/10
        hwntx = hwntx.where(hwntx>0)
        for x,y in enumerate(hwntx.season):
            if i == 0:
                cb1 = hwntx.isel(season = x).plot(ax=ax[x], cmap=cc, vmax=vmax, add_colorbar=False)
                ax[0].set_ylabel('1984-1993', fontweight='bold')

            elif i == 1:
                cb2 = hwntx.isel(season = x).plot(ax=ax[x+4], cmap=cc, vmax=vmax, add_colorbar=False)
                ax[4].set_ylabel('1994-2003', fontweight='bold')

            else:
                cb3 = hwntx.isel(season = x).plot(ax=ax[x+8], cmap=cc, vmax=vmax, add_colorbar=False)
                ax[8].set_ylabel('2004-2013', fontweight='bold')

    for i, j in enumerate(ax):
        ax[i].set_xlabel(None)

    for num in range(12):
        if num not in [0, 4, 8]:
            ax[num].set_ylabel(None)

    cb = [cb1, cb2, cb3]

    for i, j in enumerate([0.72, 0.40, 0.09]):
        cax = fig.add_axes([1, j, 0.02, 0.2])
        fig.colorbar(cb[i], cax=cax, orientation='vertical', extend='both', label=f'HWN \n ({t})')

    fig.tight_layout()
    plt.savefig(f'{path}/figures/Climatologies of Seasonal Heatwaves {t1[0]}-{t2[2]} {nameof(t)}.jpeg', bbox_inches='tight')

In [ ]:
hwn_plt(hwn_tx,'tx')

In [ ]:
hwn_plt(hwn_tn,'tn')

In [ ]:
fig, ax = plt.subplots(ncols=4, nrows=3, figsize=(10, 5), subplot_kw={'projection': crs.PlateCarree()})
ax = ax.flatten()
set_fig_params(ax)
t1 = ['1984', '1994', '2004']
t2 = ['1993', '2003', '2013']
for i, j in enumerate(t1):

    # Plot data
    vmax=3
    cc='coolwarm'

    hwn_ehf = ehf.sel(time=slice(t1[i],t2[i])).groupby('time.season').mean('time')
    # sel(time=slice(t1[i],t2[i])).groupby('time.season').sum('time')/10
    # ehf.groupby('time.season').mean('time')[2].plot(vmax=2)
    for x,y in enumerate(hwn_ehf.season):
        if i == 0:
            cb1 = hwn_ehf.isel(season = x).plot(ax=ax[x], cmap=cc, vmax=vmax, add_colorbar=False)
            ax[0].set_ylabel('ehf \n 1984-1993', fontweight='bold')

        elif i == 1:
            cb2 = hwn_ehf.isel(season = x).plot(ax=ax[x+4], cmap=cc, vmax=vmax, add_colorbar=False)
            ax[4].set_ylabel('ehf \n 1994-2003', fontweight='bold')

        else:
            cb3 = hwn_ehf.isel(season = x).plot(ax=ax[x+8], cmap=cc, vmax=vmax, add_colorbar=False)
            ax[8].set_ylabel('ehf \n 2004-2013', fontweight='bold')

for i, j in enumerate(ax):
    ax[i].set_xlabel(None)

for num in range(12):
    if num not in [0, 4, 8]:
        ax[num].set_ylabel(None)

cb = [cb1, cb2, cb3]

for i, j in enumerate([0.72, 0.40, 0.09]):
    cax = fig.add_axes([1, j, 0.02, 0.2])
    fig.colorbar(cb[i], cax=cax, orientation='vertical', extend='both', label='HWN (ehf)')

fig.tight_layout()
plt.savefig(f'{path}/figures/Climatologies of Seasonal Heatwaves {t1[0]}-{t2[2]} ehf.jpeg', bbox_inches='tight')

#### Length of the Longest Yearly Event (HWD) ####

In [ ]:
%%time

# Create subplots
fig, ax = plt.subplots(ncols=3, nrows=2, figsize=(10, 5), subplot_kw={'projection': crs.PlateCarree()})
ax = ax.flatten()
set_fig_params(ax)

vmax = 24
vmax1 = 1
vmin1 = -1
# windows = candidates.chunk({'time':20}).rolling(time=5, center=True, min_periods=1).construct('rolling_dim')
# heatwave_starts = rising_filter_dask(windows, dim='rolling_dim')
cm = 'YlOrRd'
cc = 'rainbow'

sens_slopes((ds_gtehf.where(ds_gtehf>0)).load()).plot(ax=ax[5], cmap=cc, add_colorbar=False)

# g=(HWD(tx)).groupby('time.year').sum('time').max('year')
# cb1 = g.where(g>0).plot(ax=ax[0], cmap=cm, add_colorbar=False, vmax=vmax)

hdtx = hwn_tx.groupby('time.year').sum('time').max('year')
cb1 = hdtx.where(hdtx>0).plot(ax=ax[0], cmap=cm, add_colorbar=False, vmax=vmax)
ax[0].set_title('CTx90 \n 1983-2016')

# (HWD(tn)).groupby('time.year').sum('time').max('year').plot(ax=ax[1], cmap=cm, add_colorbar=False, vmax=vmax)
hdtn = hwn_tn.groupby('time.year').sum('time').max('year')
hdtn.where(hdtn>0).plot(ax=ax[1], cmap=cm, add_colorbar=False, vmax=vmax)
ax[1].set_title('CTn90 \n 1983-2016')

# ehf.mean('time').plot(ax=ax[2], cmap=cm, add_colorbar=False, vmax=vmax)

# ds_gtehf.where(ds_gtehf>0).mean('year').plot(ax=ax[2], cmap=cm, vmax=vmax, add_colorbar=False)

ds_gtehf.where(ds_gtehf>0).max('year').plot(ax=ax[2], cmap=cm, vmax=vmax, add_colorbar=False)
ax[2].set_title('EHF \n 1983-2016')

# sens_slopes((HWD(tx.load())).groupby('time.year').sum('time').load()).plot()

cb2 = sens_slopes((HWD(tx.load())).groupby('time.year').sum('time').load()).plot(ax=ax[3], cmap=cc, add_colorbar=False, vmax = vmax1, vmin=vmin1)
sens_slopes((HWD(tn.load())).groupby('time.year').sum('time').load()).plot(ax=ax[4], cmap=cc, add_colorbar=False, vmax = vmax1, vmin=vmin1)
# sens_slopes((ds_gtehf.where(ds_gtehf>0)).load()).plot(ax=ax[5], cmap=cc, add_colorbar=False, vmax = vmax1, vmin=vmin1)

# sens_slope(tn.load()).plot(ax=ax[4], cmap=cc, add_colorbar=False, vmax = vmax1, vmin=vmin1)
# sens_slope(ehf.load()).plot(ax=ax[5], cmap=cc, add_colorbar=False, vmax = vmax1, vmin = vmin1)


ax[3].set_title('CTx90pctl \n 1983-2016')
ax[4].set_title('CTc90pctl \n 1983-2016')
ax[5].set_title('EHF \n 1983-2016')

# Add colorbar
cb = [cb1, cb2]
labels = ['HWD','Trend']
# cax2 = fig.add_axes([1,0.13,0.02,0.3])
# fig.colorbar(cb3,cax = cax2, orientation='vertical', extend='both',label='ehf-Trend')

for i, j in enumerate([0.585, 0.13]):
    cax = fig.add_axes([1, j, 0.02, 0.3])
    fig.colorbar(cb[i], cax=cax, orientation='vertical', extend='both', label=labels[i])

fig.tight_layout()
plt.savefig(path+'/figures/lenght of the longest heatwave.jpeg', bbox_inches='tight')

In [ ]:
%%time

fig, ax = plt.subplots(ncols=4, nrows=3, figsize=(10, 5), subplot_kw={'projection': crs.PlateCarree()})
ax = ax.flatten()
set_fig_params(ax)
t1 = ['1984', '1994', '2004']
t2 = ['1993', '2003', '2013']

vmax = 24
vmax1 = 1
vmin1 = -1
cm = 'YlOrRd'
cc = 'rainbow'

for i, j in enumerate(t1):

    
    hdtx = hwn_tx.groupby('time.year').sum('time').max('year')
    cb1 = hdtx.where(hdtx>0).plot(ax=ax[0], cmap=cm, add_colorbar=False, vmax=vmax)
    ax[0].set_title('CTx90 \n 1983-2016')


    # (HWD(tn)).groupby('time.year').sum('time').max('year').plot(ax=ax[1], cmap=cm, add_colorbar=False, vmax=vmax)
    hdtn = hwn_tn.groupby('time.year').sum('time').max('year')
    hdtn.where(hdtn>0).plot(ax=ax[1], cmap=cm, add_colorbar=False, vmax=vmax)
    ax[1].set_title('CTn90 \n 1983-2016')

    # ehf.mean('time').plot(ax=ax[2], cmap=cm, add_colorbar=False, vmax=vmax)

    # ds_gtehf.where(ds_gtehf>0).mean('year').plot(ax=ax[2], cmap=cm, vmax=vmax, add_colorbar=False)

    ds_gtehf.where(ds_gtehf>0).max('year').plot(ax=ax[2], cmap=cm, vmax=vmax, add_colorbar=False)
    ax[2].set_title('EHF \n 1983-2016')

    # sens_slopes((HWD(tx.load())).groupby('time.year').sum('time').load()).plot()

    cb2 = sens_slopes((HWD(tx.load())).groupby('time.year').sum('time').load()).plot(ax=ax[3], cmap=cc, add_colorbar=False, vmax = vmax1, vmin=vmin1)
    sens_slopes((HWD(tn.load())).groupby('time.year').sum('time').load()).plot(ax=ax[4], cmap=cc, add_colorbar=False, vmax = vmax1, vmin=vmin1)
    # sens_slopes((ds_gtehf.where(ds_gtehf>0)).load()).plot(ax=ax[5], cmap=cc, add_colorbar=False, vmax = vmax1, vmin=vmin1)

    # sens_slope(tn.load()).plot(ax=ax[4], cmap=cc, add_colorbar=False, vmax = vmax1, vmin=vmin1)
    # sens_slope(ehf.load()).plot(ax=ax[5], cmap=cc, add_colorbar=False, vmax = vmax1, vmin = vmin1)


    ax[3].set_title('CTx90pctl \n 1983-2016')
    ax[4].set_title('CTc90pctl \n 1983-2016')
    ax[5].set_title('EHF \n 1983-2016')

    # Add colorbar
    cb = [cb1, cb2]
    labels = ['HWD','Trend']
    # cax2 = fig.add_axes([1,0.13,0.02,0.3])
    # fig.colorbar(cb3,cax = cax2, orientation='vertical', extend='both',label='ehf-Trend')

    for i, j in enumerate([0.585, 0.13]):
        cax = fig.add_axes([1, j, 0.02, 0.3])
        fig.colorbar(cb[i], cax=cax, orientation='vertical', extend='both', label=labels[i])

    fig.tight_layout()
# plt.savefig(path+'/figures/lenght of the longest heatwave.jpeg', bbox_inches='tight')

In [ ]:
### ehf lenght of the longest heatwave days ###

fig, ax = plt.subplots(ncols=4, nrows=3, figsize=(10, 5), subplot_kw={'projection': crs.PlateCarree()})
ax = ax.flatten()
set_fig_params(ax)
t1 = ['1984', '1994', '2004']
t2 = ['1993', '2003', '2013']
for i, j in enumerate(t1):

    # Plot data
    vmax=10
    cc='coolwarm'
    hwd_ehf = ehf.sel(time=slice(t1[i],t2[i])).groupby('time.season').max('time')
    for x,y in enumerate(hwd_ehf.season):
        if i == 0:
            cb1 = hwd_ehf.isel(season = x).plot(ax=ax[x], cmap=cc, vmax=vmax, add_colorbar=False)
            ax[0].set_ylabel('ehf \n 1984-1993', fontweight='bold')

        elif i == 1:
            cb2 = hwd_ehf.isel(season = x).plot(ax=ax[x+4], cmap=cc, vmax=vmax, add_colorbar=False)
            ax[4].set_ylabel('ehf \n 1994-2003', fontweight='bold')

        else:
            cb3 = hwd_ehf.isel(season = x).plot(ax=ax[x+8], cmap=cc, vmax=vmax, add_colorbar=False)
            ax[8].set_ylabel('ehf \n 2004-2013', fontweight='bold')

for i, j in enumerate(ax):
    ax[i].set_xlabel(None)

for num in range(12):
    if num not in [0, 4, 8]:
        ax[num].set_ylabel(None)

cb = [cb1, cb2, cb3]

for i, j in enumerate([0.72, 0.40, 0.08]):
    cax = fig.add_axes([1, j, 0.02, 0.2])
    fig.colorbar(cb[i], cax=cax, orientation='vertical', extend='both', label='HWD (ehf)')

fig.tight_layout()
plt.savefig(f'{path}/figures/Climatologies of Seasonal Heatwaves {t1[0]}-{t2[2]} ehf_HWD.jpeg', bbox_inches='tight')

In [ ]:
# Your existing code to create hwn_tx

def max_consecutive_true(arr, chunk_size):
    # Create an array where True values are 1 and False values are 0
    binary_arr = arr.astype(int)

    max_consecutive = []

    for start in range(0, len(arr.time), chunk_size):
        end = min(start + chunk_size, len(arr.time))
        chunk = binary_arr.isel(time=slice(start, end))

        # Calculate the cumulative sum along the 'time' dimension in the chunk
        cumsum = chunk.cumsum(dim='time', dtype='int')

        # Find the maximum value in the cumulative sum array
        max_chunk = cumsum.max(dim='time', skipna=True)

        max_consecutive.append(max_chunk)

    max_consecutive = xr.concat(max_consecutive, dim='chunk')
    
    # Find the maximum value across all chunks
    max_overall = max_consecutive.max(dim='chunk', skipna=True)
    # dates_overall = max_consecutive.idxmax(dim='chunk', skipna=True)

    return max_overall

# Specify the chunk size (adjust as needed)
chunk_size = 1000


In [ ]:

# Calculate the maximum consecutive 'True' values for each grid point
max_consecutive = max_consecutive_true(hwn_tx, chunk_size)

In [ ]:
max_consecutive.where(max_consecutive >0).plot()

In [ ]:
hwn_txx = hwn_tx.chunk({'time':-1})

In [ ]:
# Create an array where True values are 1 and False values are 0
binary_arr = hwn_tx.astype(int)

max_consecutive = []

for start in range(0, len(hwn_tx.time), 1000):
    end = min(start + 1000, len(hwn_tx.time))
    chunk = binary_arr.isel(time=slice(start, end))

    # Calculate the cumulative sum along the 'time' dimension in the chunk
    cumsum = chunk.cumsum(dim='time', dtype='int')
    
    # print(cumsum)

#     # Find the maximum value in the cumulative sum array
    max_chunk = cumsum.max(dim='time', skipna=True)
    
    ds = cumsum.where(cumsum.max(dim='time', skipna = True))
    
#     max_consecutive.append(max_chunk)

# max_consecutive = xr.concat(max_consecutive, dim='chunk')

# # Find the maximum value across all chunks
# max_overall = max_consecutive.max(dim='chunk', skipna=True)
# # dates_overall = max_consecutive.idxmax(dim='chunk', skipna=True)



In [ ]:
ds

In [ ]:
dates = ds.idxmax(dim='time').load()

In [ ]:
d = cumsum.idxmax(dim = 'time').load()

In [ ]:
d[0][0].values.astype(np.int64)

In [ ]:
d.astype(int)#.plot()

In [ ]:
import datetime

In [ ]:
x = d.sel(longitude=-1, latitude=4, method = 'nearest').values.astype(np.int64)


In [ ]:
np.datetime64(3, 'us')

In [ ]:
np.datetime64(int(x),'us')

In [ ]:
# array = ehf.groupby('time.season').max('time')
# array.isel(longitude=array.argmax('longitude'))
# ehf.idxmax('time').values#.resample(time='M')
# ehf.groupby('time.season').max('time').argmax()
# ds = ds_gtehf.where(ds_gtehf>0).idxmax()

In [ ]:
ds_gtehf.where(ds_gtehf>0).max('year').plot()

In [ ]:
tmp_max = df_read(path,tmax,"Tmax")

In [ ]:
ds = hwn_tx.resample(time='M').sum('time').argmax('time')

In [ ]:
nan_mask = np.isnan(tx)

In [ ]:
max_consecutive_data = np.zeros_like(tx, dtype=int)

In [ ]:
# longest_heatwave_temperatures = tx['time'].where(ehf == longest_heatwave)
# longest_heatwave_period = heatwave_mask.sel(time=heatwave_mask.time[heatwave_periods == longest_heatwave_index])

In [ ]:
longest_heatwave_time = tmp_max['time'].where(tx == longest_heatwave, drop=True)

In [ ]:
# Find the start and end dates of the longest heatwave
start_date = longest_heatwave_temperatures.min().values
end_date = longest_heatwave_temperatures.max().values

In [ ]:
start_date

In [ ]:
longest_heatwave_temperatures.min('time').plot()

In [ ]:
# ds = tx.cumsum(skipna=True)

In [ ]:
ds = tx.rolling(time=5).count()

In [ ]:
##### Seasonal Averaged HWN ######
# %%time #hwntx
def hwd_plt(data,t):
    # %%time
    fig, ax = plt.subplots(ncols=4, nrows=3, figsize=(10, 5), subplot_kw={'projection': crs.PlateCarree()})
    ax = ax.flatten()
    set_fig_params(ax)
    t1 = ['1984', '1994', '2004']
    t2 = ['1993', '2003', '2013']
    for i, j in enumerate(t1):

        # Plot data
        vmax=4
        cc='coolwarm'
        # hwn_tn.groupby('time.year').sum('time').max('year')
        # hwn_tx.resample(time='M').sum('time').groupby('time.season').max('time')
        hwdtx = data.sel(time=slice(t1[i],t2[i])).resample(time='M').sum('time').groupby('time.season').max('time')
        # hwdtx = data.sel(time=slice(t1[i],t2[i])).groupby('time.season').sum('time').max()
        hwdtx = hwdtx.where(hwdtx>0)
        for x,y in enumerate(hwdtx.season):
            if i == 0:
                cb1 = hwdtx.isel(season = x).plot(ax=ax[x], cmap=cc, vmax=vmax, add_colorbar=False)
                ax[0].set_ylabel('1984-1993', fontweight='bold')

            elif i == 1:
                cb2 = hwdtx.isel(season = x).plot(ax=ax[x+4], cmap=cc, vmax=vmax, add_colorbar=False)
                ax[4].set_ylabel('1994-2003', fontweight='bold')

            else:
                cb3 = hwdtx.isel(season = x).plot(ax=ax[x+8], cmap=cc, vmax=vmax, add_colorbar=False)
                ax[8].set_ylabel('2004-2013', fontweight='bold')

    for i, j in enumerate(ax):
        ax[i].set_xlabel(None)

    for num in range(12):
        if num not in [0, 4, 8]:
            ax[num].set_ylabel(None)

    cb = [cb1, cb2, cb3]

    for i, j in enumerate([0.72, 0.40, 0.09]):
        cax = fig.add_axes([1, j, 0.02, 0.2])
        fig.colorbar(cb[i], cax=cax, orientation='vertical', extend='both', label=f'HWN \n ({t})')

    fig.tight_layout()
    plt.savefig(f'{path}/figures/Climatologies of Seasonal Heatwaves {t1[0]}-{t2[2]} {nameof(t)}_HWD.jpeg', bbox_inches='tight')

In [ ]:
hwd_plt(hwn_tx,'tx')

In [ ]:
hwd_plt(hwn_tn, 'tn')

#### Annual Sum of Contributing Heat wave Days (HWF)

In [ ]:
%%time

# Create subplots
fig, ax = plt.subplots(ncols=3, nrows=2, figsize=(10, 5), subplot_kw={'projection': crs.PlateCarree()})
ax = ax.flatten()
set_fig_params(ax)

vmax = 12
vmax1 = 0.09
vmin1 = -0.09
# windows = candidates.chunk({'time':20}).rolling(time=5, center=True, min_periods=1).construct('rolling_dim')
# heatwave_starts = rising_filter_dask(windows, dim='rolling_dim')
cm = 'YlOrRd'
cc = 'rainbow'

cb1 = (HWD(tx).sum('time') / 34).plot(ax=ax[0], cmap=cm, add_colorbar=False, vmax=vmax)
ax[0].set_title('CTx90 \n 1983-2016')

(HWD(tn).sum('time') / 34).plot(ax=ax[1], cmap=cm, add_colorbar=False, vmax=vmax)
ax[1].set_title('CTn90 \n 1983-2016')

# (HWD(ehf).sum('time') / 34).plot(ax=ax[2], cmap=cm, add_colorbar=False, vmax=vmax)
# ehf = ehf.chunk({'time':1000})
# mask = (df_read(path,tmean_accl,'Tmax')*abs(df_read(path,tmean_sig,'Tmax'))).chunk({'time':1000}).where(ehf>0,drop=True)
# mask = ehf.where(ehf>0,drop=True)
# (mask.rolling(time=3).sum()).mean('time').plot(ax=ax[2], cmap=cm, add_colorbar=False, vmax=vmax)

# ehf = df_read(path,tmean_accl,'Tmax')*abs(df_read(path,tmean_sig,'Tmax'))
(ehf.where(ehf>0,drop=True).rolling(time=3).sum()).mean('time').plot(ax=ax[2], cmap=cm, add_colorbar=False, vmax=vmax)
ax[2].set_title('EHF \n 1983-2016')

cb2 = sens_slope(tx.load()).plot(ax=ax[3], cmap=cc, add_colorbar=False, vmax = vmax1, vmin=vmin1)
sens_slope(tn.load()).plot(ax=ax[4], cmap=cc, add_colorbar=False, vmax = vmax1, vmin=vmin1)
sens_slope(ehf.load()).plot(ax=ax[5], cmap=cc, add_colorbar=False, vmax = vmax1, vmin = vmin1)


ax[3].set_title('CTx90pctl \n 1983-2016')
ax[4].set_title('CTc90pctl \n 1983-2016')
ax[5].set_title('EHF \n 1983-2016')

# Add colorbar
cb = [cb1, cb2]
labels = ['HWD','Trends']
# Add colorbars
for i, j in enumerate([0.585, 0.13]):
    cax = fig.add_axes([1, j, 0.02, 0.3])
    fig.colorbar(cb[i], cax=cax, orientation='vertical', extend='both', label=labels[i])
    
fig.tight_layout()
plt.savefig(path+'/figures/graph22.jpeg', bbox_inches='tight')

### Climatologies of Yearly Sum of participating heat wave days
   -  Average lenght of the yearly heat wave days x Average heatwave 
   -  yearly heatwave x length of yearly heat wave event / average
   
HWN x Average_lenght

In [ ]:
yearly_av_hw_tx = hwn_tx.sum('time')/33
lenght_av_hw_tx = hwn_tx.groupby('time.year').sum('time').mean('year')
hwf_tx = yearly_av_hw_tx*lenght_av_hw_tx

In [ ]:
yearly_av_hw_tn = hwn_tn.sum('time')/33
lenght_av_hw_tn = hwn_tn.groupby('time.year').sum('time').mean('year')
hwf_tn = yearly_av_hw_tn*lenght_av_hw_tn

In [ ]:
# hwf_ehf.where(hwf_ehf>0).plot(vmax=26)

In [ ]:
%%time

# tx = xr.open_dataset(f'{path1}/Tmax/chirts.Tmax.1983.2016.WA.days_p25.nc').Tmax.sortby('time').drop_duplicates('time').chunk({'time': 20})
# tn = xr.open_dataset(f'{path1}/Tmin/chirts.Tmin.1983.2016.WA.days_p25.nc').Tmin.sortby('time').drop_duplicates('time').chunk({'time': 20})

fig, ax = plt.subplots(ncols=3, nrows=1, figsize=(10, 5), subplot_kw={'projection': crs.PlateCarree()})
ax = ax.flatten()
set_fig_params(ax)

vmax = 30

# cm = 'viridis'
cc = 'coolwarm'

cb = hwf_tx.where(hwf_tx>0).plot(ax=ax[0], cmap=cc, vmax=vmax, add_colorbar=False)

# .plot(ax=ax[0], cmap=cc, vmax=vmax, add_colorbar=False)

hwf_tn.where(hwf_tn>0).plot(ax=ax[1], cmap=cc, vmax=vmax, add_colorbar=False)

# ehf.groupby('time.year').sum().mean('year').plot(ax=ax[2], cmap=cc, add_colorbar=False)
# (ehf.where(ehf>0,drop=True).rolling(time=3).sum()).mean('time').plot(ax=ax[2], cmap=cc, add_colorbar=False)
hwf_ehf.where(hwf_ehf>0).plot(ax=ax[2], cmap=cc, vmax= vmax,add_colorbar=False)
ax[2].set_title('EHF \n 1983-2016')



# sens_slope(HWF(path1, 'Tmax/tx-tx90.nc', temperature_type='Tmax', var = 'Tmax')).plot(ax=ax[3], cmap=cc)
# sens_slope(HWF(path1, 'Tmin/tn-tn90.nc', temperature_type='Tmin', var = 'Tmin')).plot(ax=ax[4], cmap=cc)
# sens_slope(HWF_ehf(path1, 'scripts/EHF.nc', var = 'Tmax')).plot(ax=ax[5], cmap=cc)

ax[0].set_title('CTx90pct \n 1983-2016')
ax[1].set_title('CTn90pct \n 1983-2016')
# ax[5].set_title('EHF \n 1983-2016')

# cb = [cb1, cb2]
# labels = ['HWF','Trends']
# # Add colorbars
# for i, j in enumerate([0.59, 0.13]):
#     cax = fig.add_axes([1, j, 0.02, 0.3])
#     fig.colorbar(cb[i], cax=cax, orientation='vertical', extend='both', label=labels[i])
cax = fig.add_axes([1,0.35,0.02,0.3])
fig.colorbar(cb, cax=cax, orientation='vertical', extend='both', label='HWF')

fig.tight_layout()
plt.savefig(path+'/figures/Yearly Number of Participating Heatwave Day.jpeg', bbox_inches='tight')

#### Decadal Analysis

###### HWN 1983-1993

In [ ]:
%%time
t1 = '2005'
t2 = '2015'
# Create subplots
fig, ax = plt.subplots(ncols=3, nrows=2, figsize=(10, 5), subplot_kw={'projection': crs.PlateCarree()})
ax = ax.flatten()
set_fig_params(ax)

# Plot data
vmax=15
cc='coolwarm'

hwdtx = hwn_tx.sel(time=slice(t1,t2)).sum('time')/10
cb1 = hwdtx.where(hwdtx>0).plot(ax=ax[0], cmap=cc, vmax=vmax, add_colorbar=False)
ax[0].set_title(f'CTx90 {t1}-{t2}')


hwdtn = hwn_tn.sel(time=slice(t1,t2)).sum('time')/10
hwdtn.where(hwdtn>0).plot(ax=ax[1], cmap=cc, vmax=vmax, add_colorbar=False)
ax[1].set_title(f'CTn90 {t1}-{t2}')

ds_gtehf.where(ds_gtehf>0).sel(year=slice(t1,t2)).mean('year').plot(ax=ax[2], cmap=cc, vmax=vmax, add_colorbar=False)
ax[2].set_title(f'EHF {t1}-{t2}')

cm = 'RdBu_r'
vm = 1

txsl = hwn_tx.groupby('time.year').sum('time').sel(year=slice(t1,t2))
# txsl = txsl.where(txsl>0).load()
cb2 = sens_slopes(txsl.where(txsl>0).load()).plot(ax=ax[3], vmax=vm, cmap=cm, add_colorbar=False)
ax[3].set_title(f'CTx90 {t1}-{t2}')


tnsl = hwn_tn.groupby('time.year').sum('time').sel(year=slice(t1,t2))
sens_slopes(tnsl.where(tnsl>0).load()).plot(ax=ax[4], vmax=vm, cmap=cm, add_colorbar=False)
ax[4].set_title(f'CTn90 {t1}-{t2}')

sens_slopes(ds_gtehf.where(ds_gtehf>0).sel(year=slice(t1,t2)).load()).plot(ax=ax[5], cmap=cm, vmax=vm, add_colorbar=False)
ax[5].set_title(f'EHF {t1}-{t2}')

# # cb = [cb1, cb2]
label = ['HWN','Trends']

# cax2 = fig.add_axes([1,0.62,0.02,0.3])
# fig.colorbar(cb3,cax = cax2, orientation='vertical', extend='both',label='ehf')
cb = [cb1, cb2]
# Add colorbars
for i, j in enumerate([0.62, 0.13]):
    cax = fig.add_axes([1, j, 0.02, 0.3])
    fig.colorbar(cb[i], cax=cax, orientation='vertical', extend='both', label = label[i])

fig.tight_layout()
plt.savefig(f'{path}/figures/Climatologies of Yearly Heatwaves {t1}-{t2}.jpeg', bbox_inches='tight')

In [ ]:
# txsl = hwn_tx.groupby('time.year').sum('time').sel(year=slice('1983''1993'))
# sens_slopes(txsl.where(txsl>0).load()).plot()
# sellonlat(hwn_tx.groupby('time.year').sum('time')).plot()#

In [ ]:
# hwn_tn.groupby('time.year').sum('time').sel(year=slice(t1,t2))#.max('year')

In [ ]:
%%time
t1 = '2005'
t2 = '2015'
# Create subplots
fig, ax = plt.subplots(ncols=3, nrows=2, figsize=(10, 5), subplot_kw={'projection': crs.PlateCarree()})
ax = ax.flatten()
set_fig_params(ax)

vmax = 24
vmax1 = 1
vmin1 = -1
# windows = candidates.chunk({'time':20}).rolling(time=5, center=True, min_periods=1).construct('rolling_dim')
# heatwave_starts = rising_filter_dask(windows, dim='rolling_dim')
cm = 'YlOrRd'
cc = 'rainbow'

sens_slopes((ds_gtehf.where(ds_gtehf>0)).sel(year=slice(t1,t2)).load()).plot(ax=ax[5], cmap=cc, add_colorbar=False)

# g=(HWD(tx)).groupby('time.year').sum('time').max('year')
# cb1 = g.where(g>0).plot(ax=ax[0], cmap=cm, add_colorbar=False, vmax=vmax)

hdtx = hwn_tx.groupby('time.year').sum('time').max('year')
cb1 = hdtx.where(hdtx>0).plot(ax=ax[0], cmap=cm, add_colorbar=False, vmax=vmax)
ax[0].set_title(f'CTx90 \n {t1}-{t2}')


# (HWD(tn)).groupby('time.year').sum('time').max('year').plot(ax=ax[1], cmap=cm, add_colorbar=False, vmax=vmax)
hdtn = hwn_tn.groupby('time.year').sum('time').sel(year=slice(t1,t2)).max('year')
hdtn.where(hdtn>0).plot(ax=ax[1], cmap=cm, add_colorbar=False, vmax=vmax)
ax[1].set_title(f'CTn90 \n {t1}-{t2}')

# ehf.mean('time').plot(ax=ax[2], cmap=cm, add_colorbar=False, vmax=vmax)

# ds_gtehf.where(ds_gtehf>0).mean('year').plot(ax=ax[2], cmap=cm, vmax=vmax, add_colorbar=False)

ds_gtehf.where(ds_gtehf>0).sel(year=slice(t1,t2)).max('year').plot(ax=ax[2], cmap=cm, vmax=vmax, add_colorbar=False)
ax[2].set_title(f'EHF \n {t1}-{t2}')

# sens_slopes((HWD(tx.load())).groupby('time.year').sum('time').load()).plot()

cb2 = sens_slopes((HWD(tx.load())).groupby('time.year').sum('time').sel(year=slice(t1,t2)).load()).plot(ax=ax[3], cmap=cc, add_colorbar=False, vmax = vmax1, vmin=vmin1)
sens_slopes((HWD(tn.load())).groupby('time.year').sum('time').sel(year=slice(t1,t2)).load()).plot(ax=ax[4], cmap=cc, add_colorbar=False, vmax = vmax1, vmin=vmin1)
# sens_slopes((ds_gtehf.where(ds_gtehf>0)).load()).plot(ax=ax[5], cmap=cc, add_colorbar=False, vmax = vmax1, vmin=vmin1)

# sens_slope(tn.load()).plot(ax=ax[4], cmap=cc, add_colorbar=False, vmax = vmax1, vmin=vmin1)
# sens_slope(ehf.load()).plot(ax=ax[5], cmap=cc, add_colorbar=False, vmax = vmax1, vmin = vmin1)


ax[3].set_title(f'CTx90pctl \n {t1}-{t2}')
ax[4].set_title(f'CTc90pctl \n {t1}-{t2}')
ax[5].set_title(f'EHF \n {t1}-{t2}')

# Add colorbar
cb = [cb1, cb2]
labels = ['HWD','Trend']
# cax2 = fig.add_axes([1,0.13,0.02,0.3])
# fig.colorbar(cb3,cax = cax2, orientation='vertical', extend='both',label='ehf-Trend')

for i, j in enumerate([0.585, 0.13]):
    cax = fig.add_axes([1, j, 0.02, 0.3])
    fig.colorbar(cb[i], cax=cax, orientation='vertical', extend='both', label=labels[i])

fig.tight_layout()
plt.savefig(f'{path}/figures/lenght of the longest heatwave {t1}-{t2}.jpeg', bbox_inches='tight')

In [ ]:
# yearly_av_hw_tx = hwn_tx.sum('time')/33
# lenght_av_hw_tx = hwn_tx.groupby('time.year').sum('time').mean('year')

In [ ]:
%%time

# tx = xr.open_dataset(f'{path1}/Tmax/chirts.Tmax.1983.2016.WA.days_p25.nc').Tmax.sortby('time').drop_duplicates('time').chunk({'time': 20})
# tn = xr.open_dataset(f'{path1}/Tmin/chirts.Tmin.1983.2016.WA.days_p25.nc').Tmin.sortby('time').drop_duplicates('time').chunk({'time': 20})

fig, ax = plt.subplots(ncols=3, nrows=1, figsize=(10, 5), subplot_kw={'projection': crs.PlateCarree()})
ax = ax.flatten()
set_fig_params(ax)

vmax = 26

# cm = 'viridis'
cc = 'Set2'

cb = hwf_tx.where(hwf_tx>0).plot(ax=ax[0], cmap=cc, vmax=vmax, add_colorbar=False)

# .plot(ax=ax[0], cmap=cc, vmax=vmax, add_colorbar=False)

hwf_tn.where(hwf_tn>0).plot(ax=ax[1], cmap=cc, vmax=vmax, add_colorbar=False)

# ehf.groupby('time.year').sum().mean('year').plot(ax=ax[2], cmap=cc, add_colorbar=False)
# (ehf.where(ehf>0,drop=True).rolling(time=3).sum()).mean('time').plot(ax=ax[2], cmap=cc, add_colorbar=False)
hwf_ehf.plot(ax=ax[2], cmap=cc, add_colorbar=False)
ax[2].set_title('EHF \n 1983-2016')



# sens_slope(HWF(path1, 'Tmax/tx-tx90.nc', temperature_type='Tmax', var = 'Tmax')).plot(ax=ax[3], cmap=cc)
# sens_slope(HWF(path1, 'Tmin/tn-tn90.nc', temperature_type='Tmin', var = 'Tmin')).plot(ax=ax[4], cmap=cc)
# sens_slope(HWF_ehf(path1, 'scripts/EHF.nc', var = 'Tmax')).plot(ax=ax[5], cmap=cc)

ax[0].set_title('CTx90pct \n 1983-2016')
ax[1].set_title('CTn90pct \n 1983-2016')
# ax[5].set_title('EHF \n 1983-2016')

# cb = [cb1, cb2]
# labels = ['HWF','Trends']
# # Add colorbars
# for i, j in enumerate([0.59, 0.13]):
#     cax = fig.add_axes([1, j, 0.02, 0.3])
#     fig.colorbar(cb[i], cax=cax, orientation='vertical', extend='both', label=labels[i])
cax = fig.add_axes([1,0.35,0.02,0.3])
fig.colorbar(cb, cax=cax, orientation='vertical', extend='both', label='HWF')

fig.tight_layout()
#plt.savefig(path+'/figures/Yearly Number of Participating Heatwave Day.jpeg', bbox_inches='tight')

In [ ]:
####Maximum Temperature
dx = hwn_tx.groupby('time.year').sum('time').idxmax('year').load().astype(int)#.plot()

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(7, 7), subplot_kw={'projection': crs.PlateCarree()})
# cb = ax.contourf(ds,levels= np.linspace(1980,2016,1), cmap='rainbow')
cb = dx.plot(ax=ax, cmap='Set2', add_colorbar=False)
ax.set_extent([-20,22,4,25])
ax.add_feature(feature.COASTLINE)
ax.add_feature(feature.BORDERS)
ax.add_feature(feature.STATES, linewidth = 0.2)
ax.set_xticks([-20,-10,0,10], crs=crs.PlateCarree())
ax.set_yticks([5,10,15,20], crs=crs.PlateCarree())
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax.xaxis.set_major_formatter(lon_formatter)
ax.yaxis.set_major_formatter(lat_formatter)
cbar = fig.colorbar(cb , orientation ='horizontal')
cbar.ax.tick_params(rotation=45)
# plt.savefig(path+'/figures/years_of_longest heatwave_monthly_pctl_1983-2016.jpeg', bbox_inches='tight')

In [ ]:
dx = hwn_tx.groupby('time.year').sum('time').max('year').load()

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(7, 7), subplot_kw={'projection': crs.PlateCarree()})
# cb = ax.contourf(ds,levels= np.linspace(1980,2016,1), cmap='rainbow')
cb = dx.where(dx>0).plot(ax=ax, cmap='Set2', add_colorbar=False)
ax.set_extent([-20,22,4,25])
ax.add_feature(feature.COASTLINE)
ax.add_feature(feature.BORDERS)
ax.add_feature(feature.STATES, linewidth = 0.2)
ax.set_xticks([-20,-16,-12,-8,-4,0,4,8,12,16,20], crs=crs.PlateCarree())
ax.set_yticks([2,4,6,8,10,12,14,16,18,20], crs=crs.PlateCarree())
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax.xaxis.set_major_formatter(lon_formatter)
ax.yaxis.set_major_formatter(lat_formatter)
cbar = fig.colorbar(cb , orientation ='horizontal')
cbar.ax.tick_params(rotation=45)
plt.grid()
# plt.savefig(path+'/figures/years_of_longest heatwave_monthly_pctl_1983-2016.jpeg', bbox_inches='tight')